In [1]:
from src.pokemon_showdown_setup import run_pokemon_showdown

pokemon_container = run_pokemon_showdown()

⏬ Pulling shreshthtuli/pokemon-showdown (if needed)...
🚀 Starting Pokemon Showdown...
   - Container: c146c788fdf6
   - UI: http://localhost:8000 


In [ ]:
from poke_env.player.player import Player
from poke_env import RandomPlayer, cross_evaluate

first_player = RandomPlayer()
second_player = RandomPlayer()

cross_evaluation = await cross_evaluate([first_player, second_player], n_challenges=5)
cross_evaluation

2025-11-10 16:55:53,844 - RandomPlayer 1 - CRITICAL - Error message received: |nametaken|RandomPlayer 1|Your authentication token was invalid.


2025-11-10 16:55:53,846 - RandomPlayer 1 - ERROR - Unhandled exception raised while handling message:
|nametaken|RandomPlayer 1|Your authentication token was invalid.
Traceback (most recent call last):
  File "c:\Users\SHRESHTH\Desktop\build-your-own-super-agents\.venv\Lib\site-packages\poke_env\ps_client\ps_client.py", line 170, in _handle_message
    raise ShowdownException("Error message received: %s", message)
poke_env.exceptions.ShowdownException: ('Error message received: %s', '|nametaken|RandomPlayer 1|Your authentication token was invalid.')
2025-11-10 16:55:53,847 - RandomPlayer 2 - CRITICAL - Error message received: |nametaken|RandomPlayer 2|Your authentication token was invalid.
2025-11-10 16:55:53,848 - RandomPlayer 2 - ERROR - Unhandled exception raised while handling message:
|nametaken|RandomPlayer 2|Your authentication token was invalid.
Traceback (most recent call last):
  File "c:\Users\SHRESHTH\Desktop\build-your-own-super-agents\.venv\Lib\site-packages\poke_env\ps_c

In [ ]:
from tabulate import tabulate

table = [["-"] + [p.username for p in players]]
for p_1, results in cross_evaluation.items():
    table.append([p_1] + [cross_evaluation[p_1][p_2] for p_2 in results])

print(tabulate(table))